In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
train_data = pd.read_csv("D:\\Not yours\\NU\Master's\\Intro to machine learning ans statistical analysis\\Project\\G-Research Crypto Forecasting\\train.csv\\train.csv")

In [46]:
import math
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
asset_details = pd.read_csv("D:\\Not yours\\NU\\Master\'s\\Intro to machine learning ans statistical analysis\\Project\\G-Research Crypto Forecasting\\asset_details.csv")
asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [26]:
crypto_df = train_data

In [27]:
crypto_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264
...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN


In [6]:
import time

# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))


In [7]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

In [29]:
# create dataframe with returns for all assets
# all_assets_2021 = pd.DataFrame([])
# for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
#   asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
#   asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
#   asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
#   #lret = log_return(asset.Close.fillna(0))[1:]
#   all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [34]:
# closedf = pd.dataframe(crypto_df[['Date','Close']])
crypto_df['Date'] = pd.to_datetime(crypto_df['timestamp'], unit='s')
# crypto_df['timestamp']

In [35]:
crypto_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,Date
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218,2018-01-01 00:01:00
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,2018-01-01 00:01:00
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643,2018-01-01 00:01:00
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922,2018-01-01 00:01:00
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264,2018-01-01 00:01:00
...,...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN,2021-09-21 00:00:00
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN,2021-09-21 00:00:00
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN,2021-09-21 00:00:00
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN,2021-09-21 00:00:00


In [38]:
crypto_df['datetime_g'] = crypto_df['Date'].dt.strftime("%Y-%m-%d")

In [41]:
maindf = crypto_df

In [43]:
closedf = maindf[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (24236806, 2)


In [44]:
close_stock = closedf.copy()

In [45]:
del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

NameError: name 'MinMaxScaler' is not defined